In [1]:
# Importing dependencies

import pandas as pd

# Clobbering any annoying pink warning boxes

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Reading data into memory and instantiating dataframe

df = pd.read_csv('../../data/processed/solar_and_daylength_data.csv', index_col = 0)

In [3]:
# Resetting index...

df.reset_index(inplace = True)

In [4]:
# ...then setting it again -- what is my deal?!

df = df.set_index(pd.to_datetime(df['Date'])).drop('Date', axis = 1)

In [5]:
# Parsing columns

df = df[['Solar Energy (kWh)', 'Inverter 1 Energy (kWh)', 'Inverter 2 Energy (kWh)', 'Daylength in decimal hours']]

In [6]:
# Taking a peek

df

,Solar Energy (kWh),Inverter 1 Energy (kWh),Inverter 2 Energy (kWh),Daylength in decimal hours
Date,,,,
2021-11-17,22.8,10.9,11.8,8.39
2021-11-18,22.0,10.6,11.4,8.37
2021-11-19,20.4,9.9,10.5,8.34
2021-11-20,21.3,10.2,11.0,8.32
2021-11-21,22.8,10.8,12.0,8.30
...,...,...,...,...
2022-11-13,0.0,0.0,0.0,8.50
2022-11-14,0.0,0.0,0.0,8.47
2022-11-15,0.0,0.0,0.0,8.45


In [7]:
# Resampling the data monthly isnt going to work, because each
# billing period is of varaied length (e.g., some bills are for 28 days,
# while some are for 32 days) of service

df.resample('M').sum()

,Solar Energy (kWh),Inverter 1 Energy (kWh),Inverter 2 Energy (kWh),Daylength in decimal hours
Date,,,,
2021-11-30,299.4,142.5,156.9,115.54
2021-12-31,473.0,196.9,275.8,248.11
2022-01-31,459.1,152.1,307.3,256.82
2022-02-28,880.4,422.0,458.3,254.34
2022-03-31,1298.6,620.2,678.4,312.81
2022-04-30,1451.0,704.1,746.8,334.16
2022-05-31,1620.8,790.5,831.0,372.34
2022-06-30,1536.0,745.7,790.1,372.93
2022-07-31,1411.6,685.9,725.5,377.85


In [8]:
# Instantiating empty dataframe

df2 = pd.DataFrame({})

# Reading energy cost data into memory

df3 = pd.read_csv('../../data/raw/energy-costs.csv')[1:-1]

# Define billing start dates

start_dates = df3['Billing Period Start'].to_list()

# and billing end dates

end_dates = df3['Billing Period End'].to_list()

# Zipping them together into a list of tuples

dates = list(zip(start_dates, end_dates))

In [9]:
dates[0]

('11/17/2021', '12/17/2021')

In [10]:
# Hacky way to reample the data based on the billing periods
# of irregular durations, with the ultimate goal of merging this dataframe
# with our `energy costs` dataframe

for i in range(len(dates)):
    
    # Yes, this is ugly and could be written better
    
    start_date = dates[i][0].split('/')[2] + '-' + dates[i][0].split('/')[0] + '-' + dates[i][0].split('/')[1]
    end_date = dates[i][1].split('/')[2] + '-' + dates[i][1].split('/')[0] + '-' + dates[i][1].split('/')[1]
    
    # Iteratively appending resampled rows to previously-empty dataframe
    
    df2 = df2.append(df.loc[start_date : end_date].sum().to_frame().T, ignore_index=True)

In [11]:
# Creating new columns

df2['Billing Period Duration'] = pd.to_datetime(pd.to_datetime(pd.Series(end_dates))) - pd.to_datetime(pd.to_datetime(pd.Series(start_dates)))
df2['Billing Period Start'] = pd.Series(start_dates)
df2['Billing Period End'] = pd.Series(end_dates)


In [12]:
# Taking a peek

df2

,Solar Energy (kWh),Inverter 1 Energy (kWh),Inverter 2 Energy (kWh),Daylength in decimal hours,Billing Period Duration,Billing Period Start,Billing Period End
0,590.2,282.2,307.8,251.94,30 days,11/17/2021,12/17/2021
1,366.8,77.1,289.7,266.70,32 days,12/18/2021,1/19/2022
2,738.0,355.5,382.6,244.05,27 days,1/20/2022,2/16/2022
3,1280.5,608.4,672.1,320.33,32 days,2/17/2022,3/21/2022
4,1323.8,641.0,682.7,312.83,28 days,3/22/2022,4/19/2022
5,1511.0,736.2,775.2,339.47,28 days,4/20/2022,5/18/2022
6,1546.6,752.0,794.8,369.80,29 days,5/19/2022,6/17/2022
7,1594.7,775.0,819.6,395.68,31 days,6/18/2022,7/19/2022
8,1123.7,547.8,576.0,330.32,27 days,7/20/2022,8/16/2022
9,1108.3,622.9,485.2,368.61,33 days,8/17/2022,9/19/2022


In [13]:
# Spot check

df.loc['2021-11-17':'2021-12-17'].sum()

Solar Energy (kWh)            590.20
Inverter 1 Energy (kWh)       282.20
Inverter 2 Energy (kWh)       307.80
Daylength in decimal hours    251.94
dtype: float64

In [14]:
# Well would ya look at that! It worked!

df4 = df2.merge(df3)
df4

,Solar Energy (kWh),Inverter 1 Energy (kWh),Inverter 2 Energy (kWh),Daylength in decimal hours,Billing Period Duration,Billing Period Start,Billing Period End,Net Usage,Net Generation,Billable kWh,...,Usage Charge,Subtotal after Adjustments,Non-Renewable Adjustment per kWh,Renewable Adjustment per kWh,Total Adjustment per kWh,Public Benefits Charge per KwH,Public Benefits Charge,California Energy Surcharge per KwH,California Energy Surcharge,Total Charge
0,590.2,282.2,307.8,251.94,30 days,11/17/2021,12/17/2021,1262,139,1123,...,131.28,189.39,0.0218,0.0214,0.0432,0.0285,5.40,0.0003,0.34,195.13
1,366.8,77.1,289.7,266.70,32 days,12/18/2021,1/19/2022,2090,44,2046,...,239.18,337.16,0.0215,0.0217,0.0432,0.0285,9.61,0.0003,0.61,347.39
2,738.0,355.5,382.6,244.05,27 days,1/20/2022,2/16/2022,1092,263,829,...,96.91,142.32,0.0218,0.0214,0.0432,0.0285,4.06,0.0003,0.25,146.63
3,1280.5,608.4,672.1,320.33,32 days,2/17/2022,3/21/2022,834,674,160,...,18.70,35.22,0.0227,0.0205,0.0432,0.0285,1.00,0.0003,0.05,36.27
4,1323.8,641.0,682.7,312.83,28 days,3/22/2022,4/19/2022,1068,595,473,...,55.29,85.33,0.0239,0.0193,0.0432,0.0285,2.43,0.0003,0.14,87.90
5,1511.0,736.2,775.2,339.47,28 days,4/20/2022,5/18/2022,1027,675,352,...,41.15,60.15,0.0086,0.0181,0.0267,0.0285,1.71,0.0003,0.11,61.97
6,1546.6,752.0,794.8,369.80,29 days,5/19/2022,6/17/2022,1380,498,882,...,103.11,129.20,0.0015,0.0172,0.0187,0.0285,3.68,0.0003,0.26,133.15
7,1594.7,775.0,819.6,395.68,31 days,6/18/2022,7/19/2022,1808,398,1410,...,164.83,200.80,0.0028,0.0159,0.0187,0.0285,5.72,0.0003,0.42,206.94
8,1123.7,547.8,576.0,330.32,27 days,7/20/2022,8/16/2022,1700,252,1448,...,169.27,205.95,0.0057,0.0130,0.0187,0.0285,5.87,0.0003,0.43,212.25
9,1108.3,622.9,485.2,368.61,33 days,8/17/2022,9/19/2022,1973,318,1655,...,193.47,252.72,0.0202,0.0098,0.0300,0.0285,7.20,0.0003,0.50,260.42


In [15]:
# Exporting CSV for downstream processing

df4.to_csv('../../data/processed/resampled_merged_data.csv')